In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.display import Audio
sound_file = '../beep-06.mp3'

import globvars as gv
import utils
import utils_tSNE as utsne
import utils_preproc as upre

import os
import sys
import mne
import json
import matplotlib.pyplot as plt
import numpy as np
import h5py
import multiprocessing as mpr
import matplotlib as mpl
import time
import gc;
import scipy.signal as sig
import pandas as pd 

import numpy as np
import utils_postprocess_HPC as postp
import pymatreader
import re

data_dir = gv.data_dir
from os.path import join as pjoin

light_only = 1
#light_only = 0
ndaysBefore = None

from dateutil import parser
# start_time = parser.parse("6 Sept 2021 19:05:15")
#start_time = parser.parse("22 Sept 2021 00:36:00")
start_time = parser.parse("10 Nov 2021 00:05:15")
end_time = parser.parse("14 Oct 2022 21:21:45")

ndaysBefore = None
#subdir = 'nointerp'
#subdir = 'nofeatsel'
subdir = 'joint2_noskip'
lookup_dir = pjoin(gv.data_dir,subdir)
recent = postp.listRecent(days=ndaysBefore, lookup_dir= lookup_dir,
                          start_time=start_time,
                                   end_time=end_time)

rawname_regex = '([S0-9,]+)'
#rawname_regex = '([S0-9]+_[a-z]+)'
re1 = re.compile('_\!_'+rawname_regex+'_.*')
re2 = re.compile('_'+rawname_regex+'_.*')
print(len(recent))
rawnames = []
for lf in recent:
    st = 0
    if light_only:
        if not lf.startswith('_!'):
            continue
    if light_only:
        r = re.match(re1,lf)
    else:
        r = re.match(re2,lf)
    if r is None:
        print('None',lf)
        continue
    cr = r.groups()[0]
    if len(cr) < 8:
        print(lf)
        continue
    rawnames += [ cr ]
rawnames = list(sorted(set(rawnames)))
print(rawnames)

import utils_postprocess_HPC as postp
prefixes = postp.listRecentPrefixes(days = ndaysBefore, light_only=light_only, 
                                    lookup_dir= lookup_dir, 
                                    custom_rawname_regex = rawname_regex, 
                                    start_time=start_time,
                                   end_time=end_time)
display(prefixes)

In [ ]:
#TODO save orig filename so that it is easier to find

In [ ]:
subdir

In [ ]:
rawnames

In [ ]:
# #label_types = ['allsep', 'trem_vs_all', 'trem_vs_quiet']
# #label_types = ['allsep']

# nraws_used_PCA = 2

# set_explicit_nraws_used_PCA = 1
# set_explicit_n_feats_PCA = 0
# set_explicit_dim_PCA = 0

# try:
#     print(output_per_raw.keys() )
# except (NameError,AttributeError) as e:
#     output_per_raw = None
    
#subdir='SSS'
#%debug
sources_type = 'parcel_aal'  # or ''
#subdir =''
r = postp.collectPerformanceInfo3(None,prefixes, nraws_used='[0-9]{2,4}',                                                                                                    
                                          sources_type = sources_type, 
                                           printFilenames=1,                                 
                                            ndays_before=ndaysBefore,
                                            use_main_LFP_chan=1,
                                             subdir=subdir, remove_large_items = 1,
                                 list_only=0, allow_multi_fn_same_prefix=0,
                                 use_light_files = light_only, rawname_regex_full=1,
                                 start_time=start_time,
                                   end_time=end_time)
#nraws_used='(10,12,20,24)'
#output_per_raw,Ximp_per_raw,gis_per_raw = r
output_per_raw,_,_ = r
print('len(output_per_raw) =', len(output_per_raw))
import gc; gc.collect()

In [ ]:
tpll = pp.multiLevelDict2TupleList(output_per_raw,4,3)
#[ tpl[:-1] for tpl in tpll]
set( [ tpl[-3:-1] for tpl in tpll] )

In [ ]:
import utils_postprocess as pp
tpll = pp.multiLevelDict2TupleList(output_per_raw,4,3)
#tpll [0][:4]  

mult_clf_output = tpll[0][-1]
all_thrs = mult_clf_output['feat_variance_q_thr'][-3:]
#thr0,thr1,thr2='0.6','0.75','0.9'
thr0,thr1,thr2='0.87','0.92','0.99'

all_LDA =  []
all_XGB = ['all_present_features','after_VF_threshold']

for thr_cur in all_thrs:
    all_XGB += [ f'best_PCA-derived_features_{thr_cur}']

perf_to_use_list = []
if len(all_LDA):
    for v in all_LDA[1:]:
        perf_to_use_list += [('LDA',all_LDA[0],v)]
for v in all_XGB[1:]:
    perf_to_use_list += [('XGB',all_XGB[0],v,'across_subj')]
for v in all_XGB[1:]:
    perf_to_use_list += [('XGB',all_XGB[0],v,'across_medcond')]
for v in all_XGB[1:]:
    perf_to_use_list += [('XGB',all_XGB[0],v,'across_subj,medcond')]    

###########################################
display(perf_to_use_list)

# to_show = [('allsep','merge_nothing','basic'), ('trem_vs_all','merge_all_not_trem','basic'),
#         ('trem_vs_2class','merge_movements','basic')]
to_show = [('trem_vs_mvt','merge_movements','trem_vs_hold&move'),
            ('trem_vs_2class','merge_movements','basic') ]
to_show += [('trem_vs_all','merge_movements', 'trem_vs_hold&move')]
          #('trem_vs_quiet','merge_nothing','trem_vs_quiet') ]
#          ('allsep','merge_nothing','basic')]
to_show = [('allsep','merge_nothing','basic')]


# warnings.simplefilter('error')
# table_info_per_perf_type, table_per_perf_type = \
#     postp.prepTableInfo2(output_per_raw, prefixes=prefixes, 
#     perf_to_use_list=perf_to_use_list)

#%debug
# import warnings
# with warnings.catch_warnings():
#warnings.simplefilter('error')
table_info_per_perf_type, table_per_perf_type = \
    postp.prepTableInfo3(output_per_raw, prefixes=prefixes, 
    perf_to_use_list=perf_to_use_list, to_show=to_show)

In [ ]:
from utils_postprocess_HPC import printOutputInfo
printOutputInfo(output_per_raw)

In [ ]:
mco = tpll[0][-1]
from utils_postprocess_HPC import computeImprovementsPerParcelGroup,plotTableInfoBrain
from utils_postprocess_HPC import filterOutputs

prefs = [pref for pref in prefixes if pref.startswith('onlyH_act_only') ]
prefs += [pref for pref in prefixes if pref.startswith('onlyH_act_exclude')]
prefs += ['onlyH_act']
outputs_filtered = postp.filterOutputs(output_per_raw,prefs=prefs, grps = ['merge_nothing'] )

for mode in ['only','exclude']:
    #%debug
    impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn, impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond =\
        computeImprovementsPerParcelGroup(outputs_filtered, 'onlyH_act', mode = mode, inv_exclude = True)
#        return impr_wrt_base_per_medcond_per_pgn, impr_per_medcond_per_pgn, impr_wrtLFP_per_medcond_per_pgn, perfs_aver_per_medcond

    for medcond in impr_per_medcond_per_pgn.keys():
        axs,crlb = plotTableInfoBrain(impr_wrt_base_per_medcond_per_pgn , medcond, mco, 
                           head_subj_ind=None, mode=mode, subdir=subdir)
        
        
        plt.close()
        #break
    #break

In [ ]:
#perf_tuple = perf_to_use_list[0]
#%debug
#for perf_tuple in perf_to_use_list[:1]:
for perf_tuple in perf_to_use_list:
    print(perf_tuple)
    postp.plotTableInfos2(table_info_per_perf_type, perf_tuple=perf_tuple, 
                          output_subdir=subdir) 
    plt.close()
    import gc;gc.collect()
    #break
#postp.plotTableInfos2(table_info_per_perf_type, perf_kind='LDA', keys = None, output_subdir=''): 

In [ ]:
#perf_to_use_list

In [ ]:
perf_to_use_list

In [ ]:
####################################### plot table infos
#perf_tuple = perf_to_use_list[0]
#%debug
#for perf_tuple in perf_to_use_list[:1]:
for perf_tuple in perf_to_use_list:
    print(perf_tuple)
    postp.plotTableInfos2(table_info_per_perf_type, perf_tuple=perf_tuple, 
                          output_subdir=subdir,use_recalc_perf=False, 
                          prefixes_sorted=prefixes, crop_rawname=slice(-3,None)) 
    plt.close()
    import gc;gc.collect()
    #break
#postp.plotTableInfos2(table_info_per_perf_type, perf_kind='LDA', keys = None, output_subdir=''): 

###########################################################
## load labels (not important where from exactly)
rncur = 'S01_off_hold'
sind_str,mc,tk  = utils.getParamsFromRawname(rncur)
sources_type='parcel_aal'
src_file_grouping_ind = 10
src_rec_info_fn = '{}_{}_grp{}_src_rec_info'.format(rncur,
                                                    sources_type,src_file_grouping_ind)
src_rec_info_fn_full = os.path.join(gv.data_dir, src_rec_info_fn + '.npz')
rec_info = np.load(src_rec_info_fn_full, allow_pickle=True)
print( list(rec_info.keys()) )

labels_dict = rec_info['label_groups_dict'][()]

In [ ]:
prefixes

In [ ]:
tpll = pp.multiLevelDict2TupleList(output_per_raw,4,3)
[ tpl[:-1] for tpl in tpll]
#set( [ tpl[-3:-1] for tpl in tpll] )

# Main feature signif plots

In [ ]:
prefixes

In [ ]:
prefixes_to_use

In [ ]:
import utils_postprocess as pp
from utils_postprocess_HPC import loadFullScores 
from matplotlib.lines import Line2D


#%debug
hh_ = None
#for prefix_cur in list(sorted( set(prefixes) - set(['all']) )):
#for prefix_cur in ['cross_freqmod_beta,gamma:HFO']:
#for prefix_cur in [ 'LFPrel_noself']:
#for prefix_cur in [ 'onlyH']:
#for prefix_cur in [ 'LFPrel_noself_onlyCon']:
#for prefix_cur in [ 'allb_beta', 'onlyH']:
#for prefix_cur in [  'allb_beta', 'allb_tremor', 'allb_gamma', 'onlyH']:
#for prefix_cur in [ 'LFPrel_noself_onlyRbcorr', 'LFPrel_noself_onlyCon', 'modSrc_self',  'onlyCBSrc', 'onlyMotorSrc']:
#for prefix_cur in [ 'LFPrel_noself_onlyBpcorr', 'LFPrel_noself']:
#for prefix_cur in [ 'all']:
#for tpl_ in [('onlyH',10), ('LFPrel_noself',25)]:
#for tpl_ in [('LFPrel_noself_onlyRbcorr',15)]:
#for tpl_ in [ ('allb_beta',15),  ('allb_gamma',15), ('allb_tremor',15) ]:
#for tpl_ in [ ('LFPrel_noself_onlyBpcorr',25)]:    
#for prefix_cur in [ 'onlyH', 'LFPrel_noself', 'allb_beta']:
#for prefix_cur in prefixes:

pref_hh_tuples = [ ('all',40)]
pref_hh_tuples = [('LFPrel_noself',35) ]
pref_hh_tuples = [ ('onlyH',10)]

subdir_short = 'joint2_noskip'
grpit_tpl = 'merge_movements','basic'

grpit_tpl = 'merge_movements','trem_vs_hold&move'




from IPython import get_ipython; ipython = get_ipython()
runstr_ = '%run -i ../run/_subrun_plot_imp_HPC.py'
ipython.magic(runstr_)

In [ ]:
import utils_postprocess as pp
from utils_postprocess_HPC import loadFullScores 
from matplotlib.lines import Line2D
collect_SHAP_outs = []

#%debug
hh_ = None
#for prefix_cur in list(sorted( set(prefixes) - set(['all']) )):
#for prefix_cur in ['cross_freqmod_beta,gamma:HFO']:
#for prefix_cur in [ 'LFPrel_noself']:
#for tpl_ in [ ('onlyH',10)]:
#for prefix_cur in [ 'LFPrel_noself_onlyCon']:
#for prefix_cur in [ 'allb_beta', 'onlyH']:
#for prefix_cur in [  'allb_beta', 'allb_tremor', 'allb_gamma', 'onlyH']:
#for tpl_ in [ 'LFPrel_noself_onlyRbcorr', 'LFPrel_noself_onlyCon', 'modSrc_self',  'onlyCBSrc', 'onlyMotorSrc']:
#for prefix_cur in [ 'LFPrel_noself_onlyBpcorr', 'LFPrel_noself']:
#for prefix_cur in [ 'all']:
#for tpl_ in [('LFPrel_noself',25), ('onlyH',10)]:
for tpl_ in [ ('LFPrel_noself_onlyRbcorr',15)]:
#for tpl_ in [ ('allb_beta',15)]:
#for prefix_cur in [  ('allb_gamma',15), ('allb_tremor',15) ]:
#for tpl_ in [ ('allb_beta',15),  ('allb_gamma',15), ('allb_tremor',15) ]:
#for tpl_ in [('LFPrel_noself_onlyBpcorr',25)]:
#for prefix_cur in [ 'onlyH', 'LFPrel_noself', 'allb_beta']:
#for prefix_cur in prefixes:
#for tpl_ in [ ('LFPrel_noself_onlyBpcorr',25)]:
    if isinstance(tpl_,str):
        prefix_cur = tpl_
    else:
        prefix_cur,hh_ = tpl_
        
    prefixes_to_use = [prefix_cur]

#     outputs_grouped = pp.groupOutputs(output_per_raw, prefixes_to_use,
#                                       ['merge_movements'],['trem_vs_hold&move'])
    outputs_grouped = pp.groupOutputs(output_per_raw, prefixes_to_use,
                                      ['merge_movements'],['basic'])
#     outputs_grouped = pp.groupOutputs(output_per_raw, prefixes_to_use,
#                                       ['merge_nothing'],['trem_vs_quiet'])
    if not len(outputs_grouped):
        continue
    postp.loadFullScores(outputs_grouped)
    #pp.printDict(outputs_grouped,1,print_leaves=1)


    use_light_file = 1
    perf_thr = 0.7

    rname_crop = slice(0,3)

    #(prefix,grp,int_type), output = u
    print(f'-- {prefix_cur}: Starting plotting for {len(outputs_grouped)} grouped outputs')
    info_cur = {}
    #sens,spec = res[rn].get(pref, (np.nan, np.nan))
    (prefix,grp,int_type), mult_clf_output = list(outputs_grouped.values())[0]

    output_subdir = ''
    rnstr = ';'.join( list( outputs_grouped.values() )[0][0] )
    out_name_plot = f'{",".join((prefix,grp,int_type) ) }_feat_signif'        
    #chnames_LFP = ['LFPR01', 'LFPR12', 'LFPR23']

    str_to_put_ = ''
    #     pvec = mult_clf_output['XGB_analysis_versions']['all_present_features']['perf_dict']['perf_aver']
    #     sens,spec,F1 = pvec
    #     str_to_put_ =  '{:.0f}%,{:.0f}%,{:.0f}%'.format(100*pvec[0],100*pvec[1],100*pvec[2])
    #     if min(sens,spec) < perf_thr:
    #         print('  Skipping due to low perf ',str_to_put_)
    #         continue
    #hh = int( 20 * len(featnames) / 270  ), 
    #%debug
    merge_Hjorth = prefix_cur != 'onlyH'
    merge_Hjorth = False
    
    Hjorth_diff_color = (prefix_cur == 'onlyH') and not merge_Hjorth
    Hjorth_diff_color = not merge_Hjorth
    
    use_same_ax = True
    #clear_yticks_for_middle_axes = False
    clear_yticks_for_middle_axes = True
    separate_by_band = 1
    
    if hh_ is None:
        hh = 12
        if prefix_cur == 'onlyH':
            if merge_Hjorth:
                hh = 5
            else:
                hh = 9
        if prefix_cur == 'LFPrel_noself_onlyBpcorr':
            hh = 9
        if prefix_cur == 'LFPrel_noself_onlyRbcorr':
            hh = 7
            if modSrc_self:
                hh = 12
        if prefix_cur == 'all':
            hh = 18
            if modSrc_self:
                hh = 25
    else:
        hh = hh_
    #outputs_grouped   # 
    #(prefix,grp,int_type), mult_clf_output = output_list[0]
    #%debug
    markers_mean = ['o','*']
    markers_max = ['x','+']
    from matplotlib.backends.backend_pdf import PdfPages
    subdir_short = 'joint2_noskip'
    figfname_full = pjoin(gv.dir_fig, subdir_short, out_name_plot + '.pdf')
    figfname_full = figfname_full.replace('&','_AND_')
    pdf= PdfPages(figfname_full )        
    axs = None
    legend_elements = []
    for i,og in enumerate(outputs_grouped.items() ):
        rn = og[0][0]
        (prefix,grp,int_type), mult_clf_output = og[1]
        ogg = dict( [og] )
        if not use_same_ax:
            axs = None
        axs, collect_SHAP_outs_cur = postp.plotFeatSignifSHAP_list(pdf=None,
                                 outputs_grouped=ogg, fshs='XGB_Shapley',                                  
                                 figname_prefix=prefix, roi_labels=labels_dict['all_raw'],
                                 body_side='left', chnames_LFP=None, 
                                 hh = hh, ww = 7,
                                 tickfontsize = 10, markersize=8,
                                 suptitle = str_to_put_, use_best_LFP=False,
                                 suptitle_fontsize=20, show_bias=1, show_max = True, 
                                 show_std = False, average_over_subjects = True,
                                 merge_Hjorth = merge_Hjorth,alpha=0.8, 
                                      use_full_scores=True, show_abs_plots=1, reconstruct_from_VIF=1,
                                    marker_mean=markers_mean[i] ,marker_max = markers_max[i],
                                    Hjorth_diff_color = Hjorth_diff_color,
                                    axs=axs, grand_average_per_feat_type=0,
                                        separate_by_band=separate_by_band);     
        legend_alpha = 0.25
        legend_elements += [Line2D([0], [0], marker=markers_mean[i], color='orange', lw=0, 
                               label=f'mean {rn[-3:]}', markerfacecolor='orange', markersize=15,alpha=legend_alpha) ]
        legend_elements += [Line2D([0], [0], marker=markers_max[i], color='orange', lw=0,
                               label=f'max {rn[-3:]}', markerfacecolor='orange', markersize=15,alpha=legend_alpha) ]
    
        if not use_same_ax:
            pdf.savefig();    plt.close()       
            axs[0,0].set_title(axs[0,0].get_title() + f' {rn[:-3]}')
            
            if clear_yticks_for_middle_axes:
                for i in range(len(axs) ):
                    for j in range( 1,4 ):
                        axs[i,j].set_yticklabels([])
                        #axs[i,j].set_yticks([])
                    for j in range(4):    
                        axs[i,j].grid()
        collect_SHAP_outs += collect_SHAP_outs_cur    
    
    if use_same_ax:        
        axs[0,0].set_title( str(axs[0,0].get_title() ) + f' {rn[:-3]}*')
        axs[0,1].legend(handles= legend_elements, loc='lower right')
        
        if clear_yticks_for_middle_axes:
            for i in range(len(axs) ):
                for j in range (1,4):
                    axs[i,j].set_yticklabels([])
                    #axs[i,j].set_yticks([])
                for j in range (4):    
                    axs[i,j].grid()
            
        plt.tight_layout()
        pdf.savefig();    plt.close()       
            
    cax,clrb = postp.plotConfmats(outputs_grouped, best_LFP=0, ww=9,hh=9)
    kind = ';'.join( list( outputs_grouped.values() )[0][0] )
    figname = ','.join( [k[0] for k in outputs_grouped.keys()] ) + f'_confmats_{kind}.pdf'
    kind = ';  '.join( list( outputs_grouped.values() )[0][0] )
    plt.suptitle(kind)
    pdf.savefig();    plt.close();    pdf.close()
    print('saved to ',figname)

#             break+
#         break
#     break

In [ ]:
subdir

In [ ]:
#subdir = 'per_subj_best_LFP'

pref_hh_tuples = [ ('all',40)]

pref_hh_tuples = [ ('onlyH',10)]
EBM_feat_subsets = ['VIFsel']
#EBM_feat_subsets = ['all']


#subdir_short = 'joint_noskip'
grpit_tpl = 'merge_movements','basic'

#%debug
from IPython import get_ipython; ipython = get_ipython()
runstr_ = '%run -i ../run/_subrun_plot_imp_EBM_HPC.py'
ipython.magic(runstr_)

In [ ]:
len(outputs_grouped)

In [ ]:
dd = 'bias/6.1694/2.4873'
np.sum( np.array(list(dd)) == '/' ) 


In [ ]:
import gc;gc.collect()

In [ ]:
common = set(mult_clf_output['VIF_truncation']['colinds_bad_VIFsel'])
#print(common)
for i,og in enumerate(outputs_grouped.items() ):
    rn = og[0][0]
    (prefix,grp,int_type), mult_clf_output = og[1]
    VIFtr = mult_clf_output['VIF_truncation']
    badinds = VIFtr['colinds_bad_VIFsel']
    print(len(badinds), badinds)
    common = common & set(badinds)
    
print(len(common),sorted(common,reverse=1) )

In [ ]:
d[(rn,prefix,grp,it)]

In [ ]:
VIFtr.keys()

In [ ]:
VIFtr['VIFsel_linreg_objs'][0].__dict__.keys()

In [ ]:
d = {}
for rn,prefix,grp,it,mult_clf_output in tpll:
    VIFtr = mult_clf_output['VIF_truncation']
    dd = dict( VIFtr.items() )
    #del dd['VIFsel_linreg_objs']
    #dd[]
    #VIFtr['VIFsel_linreg_objs'][0]
    d[(rn,prefix,grp,it)] = dd
#('S01,S02,S04,S05,S07_off', 'allb_beta', 'merge_movements', 'basic')
VIF_info_fname = pjoin(gv.data_dir,subdir_short,'VIF_info.npz')
print(VIF_info_fname)
np.savez(VIF_info_fname,VIF_info = d)

In [ ]:
# (rn_,prefix,grp,int_type,fsh,featnames_nice,label_str,\
#      scores_pre_class_curlab, feat_imp_stats ) = collect_SHAP_outs[0]


runMLvars_pso = []
for cso in collect_SHAP_outs:
    (rn_,prefix,grp,int_type,fsh,featnames_nice,label_str,\
         scores_pre_class_curlab, feat_imp_stats ) = cso 
    
    mult_clf_output = output_per_raw[rn_][prefix][grp][int_type]
    #assert (prefix,grp,int_type) == (prefix_,grp_,int_type_)    
    cmd = mult_clf_output['cmd']
    print(cmd[0])

    ####################

    s = '%run -i ../run/run_ML.py '
    for a,b in cmd[0]:
        s += f'{a} {b} '
    print(s)
    from IPython import get_ipython; ipython = get_ipython()

    #%run -i ../run/run_ML.py -r S01_on_hold,S01_on_move,S02_on_hold,S02_on_move,S04_on_hold,S04_on_move,S05_on_hold,S05_on_move,S07_on_hold,S07_on_move --param_file ML_joint_one_LFP_HPC.ini --groupings_to_use merge_movements --int_types_to_use basic --feat_types H_act,H_mob,H_compl --prefix onlyH --SLURM_job_id 114703_22 --calc_MI 0

    ipython.magic(s + ' --exit_after artif_processed --show_plots 0 --do_cleanup 0')
    anndict_per_intcat_per_rawn_out = anndict_per_intcat_per_rawn

    runMLvars = {}
    runMLvars['featnames_nice'] = featnames_nice
    runMLvars['featnames'] = featnames
    runMLvars['X_pri'] = X_pri
    runMLvars['rawnames'] = rawnames
    runMLvars['Xtimes_pri'] = Xtimes_pri
    runMLvars['wbd_pri'] = wbd_pri
    runMLvars['anndict_per_intcat_per_rawn'] = anndict_per_intcat_per_rawn
    runMLvars['new_main_side_pri'] = new_main_side_pri
    runMLvars['roi_labels'] = roi_labels
    runMLvars['srcgrouping_names_sorted'] = srcgrouping_names_sorted
    runMLvars['rawtimes_pri'] = rawtimes_pri
    
    runMLvars_pso += [runMLvars]

    rng = slice(0,10)
    
    # TODO: right now these are not correct scores (perhaps) because of the way they are averaged
    featis = np.argsort(np.abs(scores_pre_class_curlab) )
    featis = featis[::-1]
    featis = featis[rng] #np.random.randint( len(featnames),size=20)

    # anns, anns_pri, times_concat, dataset_bounds, wbd_merged = utsne.concatAnns(rawnames,
    #                                                           rawtimes_pri, crop=(crop_start,crop_end),
    #                                                           side_rev_pri = side_switch_happened_pri,
    #                                                          wbd_pri = wbd_pri, sfreq=sfreq, ret_wbd_merged=1)
    # print('times_concat end {} wbd end {}'.format(times_concat[-1] * sfreq, wbd_merged[1,-1] ) )

    # ivalis = utils.ann2ivalDict(anns)
    # ivalis_tb_indarrays_merged = \
    #     utils.getWindowIndicesFromIntervals(wbd_merged,ivalis,
    #                                         sfreq,ret_type='bins_contig',
    #                                         ret_indices_type = 'window_inds',
    #                                         nbins_total=nbins_total )

    # anndict_per_intcat = f['anndict_per_intcat'][()]
    # anndict_per_intcat_per_rawn[rawn] = anndict_per_intcat
    # bindict_per_rawn[rawn] = upre.markedIntervals2Bins(anndict_per_intcat,
    #                                                    rawtimes,sfreq,wbd=wbd)
    xlim = (-4,8)
    #xlim = None
    figname = 'hists_'+ f'{(rn_,prefix,grp,int_type,label_str,fsh)}_rng={rng}.pdf'
    print('start plotting ',figname)
    postp.plotFeatHists(rawnames,featnames_nice,featis,X_pri,Xconcat, bindict_per_rawn,
                      ivalis_tb_indarrays_merged, xlim_common = xlim,nbins=25, savefig = 0)
    
    
    plt.savefig(pjoin(gv.dir_fig, subdir_short, figname))
    plt.close()


In [ ]:
import gc; gc.collect()

In [ ]:
(prefix_,grp_,int_type_)

## get info from run_genfeats (for a fixed rn_)

In [ ]:
mainLFPchan_per_rawn = {}
for runMLvars in runMLvars_pso:
    for rawn in runMLvars['rawnames']:
        subj,medcond,task  = utils.getParamsFromRawname(rawn)
        mainLFPchan_cur = best_LFP_info[subj][f'{best_LFP_prefix},merge_movements,basic']['best_LFP']
        #print(rawn,subj,mainLFPchan_cur)
        mainLFPchan_per_rawn[rawn] = mainLFPchan_cur

In [ ]:



rns = list(output_per_raw.keys() )
genfeat_info_per_rn = {}
for rn_to_plot in rns:
    mult_clf_output = output_per_raw[rn_][prefix_][grp][int_type]
    ML_info = mult_clf_output['info']
    fn_feat = ML_info['fname_feat_full_pri']

    dat_out_pri = []
    dat_lfp_hires_out_pri = []
    times_hires_out_pri = []
    times_out_pri = []
    subfeature_order_out_pri = []
    subfeature_order_lfp_hires_out_pri = []
    extdat_out_pri = []
    rawnames_out = []
    emgdat_pri = []
    
    for i,fn_feat_cur in enumerate(fn_feat):
        ffeat = np.load(fn_feat_cur, allow_pickle=True )
        ss = ' '.join([' '.join(list(tpl)) for tpl in  ffeat['cmd'][0]] )
        s = '%run -i ../run/run_genfeats.py ' + ss + ' --exit_after prescale_data'
        print(s)
        ipython.magic(s)

        dat_out_pri              += dat_pri
        dat_lfp_hires_out_pri    += dat_lfp_hires_pri
        times_hires_out_pri      += times_hires_pri
        times_out_pri            += times_pri
        subfeature_order_out_pri += subfeature_order_pri
        subfeature_order_lfp_hires_out_pri += subfeature_order_lfp_hires_pri
        extdat_out_pri           += extdat_pri
    
    import re
    for fn in fn_feat:
        r = re.match('.*(S[0-9]+_[a-z]+_[a-z]+)_.*',fn).groups()[0]
        #rint(r)
        rawnames_out += [r]


    emgraw_pri = []
    for rawi,rawn in enumerate(ML_info['rawnames']):
        emgraw = mne.io.read_raw_fif ( pjoin(gv.data_dir,rawn + '_emg_rectconv.fif') )
        chns_cur = gv.EMG_per_hand[main_side_pri[rawi]]
        emgraw.pick_channels(chns_cur)
        emgraw.resample(256)
        #emgraw_pri += [emgraw]
        emgdat_pri += [emgraw.get_data()]
        assert emgdat_pri[-1].shape[1] == len(times_out_pri[rawi])

    from featlist import replaceMEGsrcChnamesParams
    subfeature_order_newsrcgrp_pri = ML_info.get('chnames_newsrcgrp_pri')
    if subfeature_order_newsrcgrp_pri is None:
        subfeature_order_newsrcgrp_pri = [0] * len(rawnames_out)
        for rawi in range(len(runMLvars['rawnames'])):
            subfeature_order_newsrcgrp_pri[rawi] = \
                replaceMEGsrcChnamesParams(subfeature_order_out_pri[rawi], 0,9, '.*', 0)
            
            
    d = {}
    d['dat_out_pri'] = dat_out_pri
    d['dat_lfp_hires_out_pri'] = dat_lfp_hires_out_pri
    d['times_hires_out_pri'] = times_hires_out_pri
    d['times_out_pri'] = times_out_pri
    d['subfeature_order_out_pri'] = subfeature_order_out_pri
    d['subfeature_order_lfp_hires_out_pri'] = subfeature_order_lfp_hires_out_pri
    d['extdat_out_pri'] = extdat_out_pri
    d['rawnames_out'] = rawnames_out
    d['emgdat_pri'] = emgdat_pri
    d['subfeature_order_newsrcgrp_pri'] = subfeature_order_newsrcgrp_pri
    genfeat_info_per_rn[rn_to_plot] = d

# best_LFP_info_fname = pjoin(gv.data_dir, 'best_LFP_info.json')
# with open(best_LFP_info_fname, 'r') as f:
#     best_LFP_info = json.load(f)



display(mainLFPchan_per_rawn)

In [ ]:
rn_,prefix,grp,int_type,fsh,featnames_nice_VIF,label_str,\
         scores_pre_class_curlab, feat_imp_stats

In [ ]:
# plot time traces

for collect_SHAP_outs_ind in range(len(collect_SHAP_outs)):
    #collect_SHAP_outs_ind = 0
    (rn_,prefix,grp,int_type,fsh,featnames_nice_VIF,label_str,\
         scores_pre_class_curlab, feat_imp_stats ) = collect_SHAP_outs[collect_SHAP_outs_ind]
    print(rn_)
    
    runMLvars = runMLvars_pso[collect_SHAP_outs_ind]
    d = genfeat_info_per_rn[rn_]
    
#     ##########################  just for now to avoid recalc of genfeats
#     if rn_.endswith('_off'):
#         continue
#     ###########################    
        
    featis = np.argsort(np.abs(scores_pre_class_curlab) )
    featis = featis[::-1]
    
    mult_clf_output = output_per_raw[rn_][prefix][grp][int_type]
    VIF_truncation = mult_clf_output['VIF_truncation']
    colinds_good_VIFsel = VIF_truncation['colinds_good_VIFsel']
    featnames_VIFsel = runMLvars['featnames'][colinds_good_VIFsel]
    featnames_best_LFP = runMLvars['featnames']

    from globvars import gp
    beh_states_to_shade = [f'{it}_{main_side_let}' for it in gp.int_types_basic]

    # TODO: right now these are not correct scores (perhaps) because of the way they are averaged
    #%debug
    rng = slice(0,8)
    #rng = slice(0,2)
    rng_feats = slice(0,2)
    #rng_feats = slice(0,1)
    from plots import plotFeatsAndRelDat
    
    axs = plotFeatsAndRelDat(runMLvars['rawnames'][rng], 
                             featnames_VIFsel[featis[rng_feats]], d['dat_out_pri'],
                       d['subfeature_order_out_pri'],
                    runMLvars['X_pri'],[featnames_best_LFP]*len(d['rawnames_out']),
                    d['times_out_pri'],
                    runMLvars['Xtimes_pri'],
                    d['subfeature_order_newsrcgrp_pri'], runMLvars['wbd_pri'],
                    dat_hires_pri=d['dat_lfp_hires_out_pri'],
                    chnames_all_hires_pri = d['subfeature_order_lfp_hires_out_pri'],
                    times_hires_pri=d['times_hires_out_pri'],
                    anndict_per_intcat_per_rawn=runMLvars['anndict_per_intcat_per_rawn'], 
                    sfreq=sfreq, mainLFP_per_rawn = mainLFPchan_per_rawn,
                    roi_labels=roi_labels, srcgrouping_names_sorted=['all_raw']*10,
                            main_side_let = main_side_let, ww=6,hh=3,
                            beh_states_to_shade = beh_states_to_shade, 
                             extdat_pri=d['emgdat_pri'])

    plt.tight_layout()
    plt.savefig(pjoin(gv.dir_fig,'joint',f'good_feats_dyn_{(rn_,prefix,grp,int_type,label_str,fsh)}_{rng}.pdf'))
    plt.close()

In [ ]:
len(collect_SHAP_outs)

# interpretML

In [ ]:
#for prefix_cur in prefixes:
#for prefix_cur in prefixes[:3]:
#for prefix_cur in prefixes:
for prefix in ['onlyMotorSrc']:
    prefixes_to_use = [prefix_cur]

    outputs_grouped = pp.groupOutputs(output_per_raw, prefixes_to_use,
                                      ['merge_movements'],['trem_vs_hold&move'])
    print('      ',prefix_cur)
    for og in outputs_grouped.items():
        rn = og[0]
        (prefix,grp,int_type), mult_clf_output = og[1]
        assert prefix == prefix_cur
    
        filename_fullsize = mult_clf_output['filename_full']
        from pathlib import Path
        pfsz = Path(filename_fullsize)
        filename_fullsize = pjoin(pfsz.parents[0], pfsz.name[2:])
        #    finfo = os.stat( filename_fullsize )
        #    print(finfo.st_size / (1024**2))
        f = np.load(filename_fullsize,allow_pickle=True)

        results_cur =  f['results_cur'][()]

        #results_cur.keys()

        EBM = results_cur['featsel_per_method']['interpret_EBM']        
        #scores

        clf_dict = EBM['info_per_cp'][('trem_L', 'hold_L&move_L')]
        scores = clf_dict['scores']; 
        #print(len(results_cur['feature_names_filtered']), len(EBM['feature_indices_used']) )
        #print('len(scores) = ',len(scores) )
        explainer = clf_dict['explainer']
        
        print((rn,grp,int_type), utsne.sprintfPerfs(clf_dict['perf'] ) )

        del f
        del results_cur
        gc.collect()

In [ ]:
#       LFPrel_noself
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 63.70%,97.30%,73.97%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 54.75%,93.76%,65.06%
#        LFPrel_noself_onlyBpcorr
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 53.09%,96.59%,64.78%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 49.71%,93.22%,60.30%
#        LFPrel_noself_onlyCon
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 42.19%,94.97%,53.35%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 30.51%,92.18%,41.29%
#        LFPrel_noself_onlyRbcorr
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 48.94%,97.15%,62.01%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 35.56%,92.61%,46.82%
#        allb_beta
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 64.64%,95.34%,72.04%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 55.94%,90.69%,63.34%
#        cross_freqmod_beta,gamma:HFO
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 18.93%,97.11%,29.55%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 26.17%,94.16%,37.62%
#        cross_freqmod_tremor,beta:HFO
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 40.86%,95.38%,52.56%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 35.36%,92.31%,46.41%
#        cross_freqmod_tremor,gamma:beta
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 39.69%,96.07%,52.15%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 27.67%,94.66%,39.64%
#        modLFP
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 35.31%,95.57%,47.26%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 4.50%,98.84%,8.41%
#        modSrc
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 72.72%,97.23%,80.13%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 62.01%,93.29%,70.18%
#        modSrc_self
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 63.11%,95.55%,71.22%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 55.49%,90.39%,62.74%
#        onlyCBSrc
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 53.38%,95.51%,63.69%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 22.85%,94.62%,33.94%
#        onlyH
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 62.04%,95.14%,69.86%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 51.43%,91.03%,60.01%
#        onlyMotorSrc
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 74.52%,97.34%,81.46%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 63.63%,94.94%,72.74%
#        onlyRestSrc
# (('S01,S02,S04,S05,S07_on',), 'merge_movements', 'trem_vs_hold&move') 66.35%,96.35%,74.51%
# (('S01,S02,S04,S05,S07_off',), 'merge_movements', 'trem_vs_hold&move') 60.19%,92.75%,68.41%

In [ ]:
(rn_,prefix,grp,int_type,fsh,featnames_nice,label_str,\
     scores_pre_class_curlab, feat_imp_stats ) = collect_SHAP_outs[0]

print(rn_,prefix,grp,int_type)
mult_clf_output = output_per_raw[rn_][prefix][grp][int_type]

In [ ]:
mult_clf_output = output_per_raw['S01,S02,S04,S05,S07_off']['onlyH'][grp][int_type]

In [ ]:
mult_clf_output = output_per_raw['S01,S02,S04,S05,S07_on']['onlyMotorSrc'][grp][int_type]

In [ ]:
print(range(0,6))

In [ ]:
[4  for d in [4,5] if d in [4,5]]

In [ ]:
help(plotFeatsAndRelDat)

In [ ]:
del plotFeatsAndRelDat

In [ ]:
import plots
import importlib as il; il.reload(plots)

In [ ]:
runMLvars['wbd_pri'][0]

In [ ]:
extdat_out_pri[0].shape

In [ ]:
from globvars import gp
beh_states_to_shade = [f'{it}_{main_side_let}' for it in gp.int_types_basic]

# TODO: right now these are not correct scores (perhaps) because of the way they are averaged
#%debug
rng = slice(0,8)
#rng = slice(0,2)
rng_feats = slice(0,2)
#rng_feats = slice(0,1)
from plots import plotFeatsAndRelDat
axs = plotFeatsAndRelDat(runMLvars['rawnames'][rng], 
                         featnames_VIFsel[featis[rng_feats]], dat_out_pri,
                   subfeature_order_out_pri,
                runMLvars['X_pri'],[featnames_best_LFP]*len(rawnames_out),
                times_out_pri,
                runMLvars['Xtimes_pri'],
                subfeature_order_newsrcgrp_pri, runMLvars['wbd_pri'],
                dat_hires_pri=dat_lfp_hires_out_pri,
                chnames_all_hires_pri = subfeature_order_lfp_hires_out_pri,
                times_hires_pri=times_hires_out_pri,
                anndict_per_intcat_per_rawn=runMLvars['anndict_per_intcat_per_rawn'], 
                sfreq=sfreq, mainLFP_per_rawn = mainLFPchan_per_rawn,
                roi_labels=roi_labels, srcgrouping_names_sorted=['all_raw']*10,
                        main_side_let = main_side_let, ww=6,hh=3,
                        beh_states_to_shade = beh_states_to_shade, 
                         extdat_pri=emgdat_pri)

plt.tight_layout()
plt.savefig(pjoin(gv.dir_fig,'joint',f'good_feats_dyn_{(rn_,prefix,grp,int_type,fsh)}_{rng}.pdf'))
plt.close()

In [ ]:
best_feat_name

In [ ]:
print( utsne.sprintfPerfs(clf_dict['perf'] ) )
print( clf_dict['confmat_normalized'] )

In [ ]:
np.argmax(scores)

In [ ]:
assert len( explainer.feature_names ) == len(scores)
best_feat_name = explainer.feature_names [ np.argmax(scores)]
print(best_feat_name)
sortinds = np.argsort(scores)
last_ind = np.where([ explainer.feature_names[ind].find(' x ') < 0 \
                     for ind in  sortinds])[0][-1]
last_ind = sortinds[last_ind]
best_feat_name_no_int = explainer.feature_names[last_ind]
print(best_feat_name_no_int)

In [ ]:
sortinds[-1]

In [ ]:
 len(scores), last_ind

In [ ]:
explainer.feature_names[last_ind], m[last_ind], best_feat_name_no_int.find(' x ')

In [ ]:
np.where(m)[0][-1]

In [ ]:
m = [ explainer.feature_names[ind].find(' x ') < 0 \
                     for ind in np.argsort(scores) ]
print(m[344])

In [ ]:
print( explainer.__dict__.keys() )
# print( explainer.explanation_type, explainer.name, 
#      explainer.selector, explainer.feature_types)

In [ ]:
from interpret import preserve
preserve(explainer, best_feat_name_no_int)

In [ ]:
preserve(explainer, best_feat_name)

In [ ]:
explainer.visualize()

In [ ]:
# from interpret import show
# show(explainer)  # does not work if open saved

In [ ]:
# Trying to get a better distr fit for Hjorth (no success)

In [ ]:
len(featnames_nice), len(featnames_VIFsel)

In [ ]:
np.array(featnames_nice)[featis[-2:]]

In [ ]:
featnames[featis[-2:]]

In [ ]:
roi_labels['all_raw'][30]

In [ ]:
len(featnames_nice), len(featnames)

In [ ]:
featnames_nice[-5:], featnames[-5:]

In [ ]:
pjoin(subdir,'good_feats_dyn.pdf')

In [ ]:
# from plots import plotFeatsWithEverything
# #%debug
# plotFeatsWithEverything(dat_pri, rawtimes_pri, X_pri, 
#                             Xtimes_pri, dat_lfp_hires_pri, times_hires_pri,
#                             rawnames,
#                             subfeature_order_pri, subfeature_order_newsrcgrp_pri,
#                             subfeature_order_lfp_hires_pri,
#                             anndict_per_intcat_per_rawn,
#                             featnames, wbd_pri,
#                             sfreq, raw_perband_flt_pri, raw_perband_bp_pri,
#                             scale_data_combine_type,
#                             stats_multiband_flt, stats_multiband_bp,
#                             test_plots_descr, special_chns,
#                            fband_names_inc_HFO )

In [ ]:
np.min(Xconcat), np.max(Xconcat_to_fit)

In [ ]:
dd0 = Xconcat[:,featis[0]]
#dd = dd[ dd< np.quantile(dd,0.95)]
dd = np.exp(dd0) 
#dd = 1/dd
dd = np.log( dd - np.min(dd) *.99 )
#dd = dd[ dd< np.quantile(dd,0.99)]
plt.hist(dd,bins=250,alpha=0.5,label='new');

plt.hist(dd0,bins=250,alpha=0.5,label='old');
plt.legend(loc='upper right')

In [ ]:
plt.hist(dd0,bins=250)

In [ ]:
np.min(dd)

In [ ]:
h

In [ ]:
help(dist.pdf)

In [ ]:
f

In [ ]:
import scipy.stats
qm = np.quantile(dd,0.99)
y = dd [ dd < qm]
#y = np.exp(y)
size = len(y)
x = np.arange(np.min(y),np.max(y),0.5)
#y = 0.01 -np.min(dd) + dd
#x = np.arange(0.01,55,0.5)
#y = scipy.int_(np.round_(scipy.stats.vonmises.rvs(5,size=size)*47))
h = plt.hist(y, bins=x)
binh,binloc,barcont = h

# beta, rayleigh look ok
#ten best
# dist_names = ['cauchy',
#  'dweibull',
#  'foldcauchy',
#  'dgamma',
#  't',
#  'tukeylambda',
#  'fisk',
#  'weibull_min', 'erlang']

#'loglaplace',
 
dist_names = ['alpha', 'cauchy',  'dgamma', 'dweibull', 'erlang', 'exponweib',
              'genlogistic',  'foldcauchy',  'genextreme']
#  'fisk',  'fatiguelife',  'gumbel_r',  'gengamma','invweibull',
#  'invgamma',  'mielke',  'powerlognorm',  'nct',
#  'johnsonsu',  'lognorm',  'invgauss',  'loglaplace',
#  'recipinvgauss',  'pearson3',  'laplace', 't',  'tukeylambda',
#  'hypsecant',  'weibull_min',  'logistic',  'kstwobign',
#               'nakagami',  'rayleigh',  'rice',  'maxwell']
dist_names = ['beta']
dist_names = ['levy_stable']

do_plot = 1
d_per_name = {}
pdf_fit_per_name = {}

for dist_name in dist_names:
    dist = getattr(scipy.stats, dist_name)
    params = dist.fit(y)
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]
    if arg:
        pdf_fitted = dist.pdf(binloc, *arg, loc=loc, scale=scale) * size
    else:
        pdf_fitted = dist.pdf(binloc, loc=loc, scale=scale) * size
    if do_plot and np.max(pdf_fitted) < np.max(binh) * 3:
        plt.plot(binloc, pdf_fitted, label=dist_name)
    plt.xlim(-6,20)
    pdf_fit_per_name[dist_name] = pdf_fitted
    
    discrep = np.sum(np.abs( binh-pdf_fitted[:-1] ) ) / len(binh)
    d_per_name[dist_name] = discrep
plt.legend(loc='upper right')
plt.show()

In [ ]:
arg

In [ ]:
help(sorted)

In [ ]:
stl = sorted( list( d_per_name.items() ), key = lambda x: x[1] )
stl_ext = [ (dn,v,pdf_fit_per_name[dn][0]) for  (dn, v) in stl]
display(stl_ext)

In [ ]:
list( set( ['exponpow', 'f', 'fatiguelife', 'fisk', 'foldcauchy', 'foldnorm', 
               'frechet_r', 'frechet_l', 'genlogistic', 'genpareto', 'genexpon', 
               'genextreme', 'gausshyper', 'gamma', 'gengamma', 'genhalflogistic', 
               'gilbrat', 'gompertz', 'gumbel_r', 'gumbel_l', 'halfcauchy'] ) - \
set( ['frechet_r',
 'genhalflogistic',
 'foldnorm',
 'gilbrat',
 'gompertz',
 'genpareto',
 'genexpon',
 'halfcauchy',
 'exponpow',
 'gamma',
 'frechet_l',
 'f',
 'gausshyper',
 'gumbel_l'] ) )

In [ ]:
list(list(zip(*stl))[0])[:10]

In [ ]:
list(list(zip(*stl))[0])[-14:]

In [ ]:
list(list(zip(*stl))[0])[-10:]

In [ ]:
len(binh), len(pdf_fitted)

In [ ]:
plt.plot(binh)
plt.plot(pdf_fitted[:-1])

In [ ]:
help(plt.subplots_adjust)